# Visualize Lidar pointclouds and Radar pointclouds


## Initialization

If got Error rendering output item using 'jupyter-ipywidget-renderer' in vscode, set `Jupyter: Widget Script Sources` in settings.

In [ ]:
try:
    import rerun as rr
    import matplotlib.pyplot as plt

except ImportError:
  print("The modules are missing. Installing now via pip...")
  # These versions of numpy and pandas-gbq are compatible with the rest of the colab environment
  %pip install rerun-sdk rerun-notebook matplotlib
  print('Installation completed. The runtime needs to be restarted. Stopping now.')
  import os
  os.kill(os.getpid(), 9)

import os
import numpy as np
from tqdm import tqdm
import re
from scipy.spatial.transform import Rotation as R, Slerp
import struct


hesai_fieldtypecodes = ('f', 'f', 'f', 'f', 'd', 'H')
hesai_unpack_numbytes = (4, 4, 4, 4, 8, 2)

def read_pcd_file(filename):
    """Read a PCD file and return (fields, points: list of lists)."""
    with open(filename, 'rb') as file:
        fields = []
        # Parse header
        while True:
            line = file.readline().decode('utf-8')
            if 'FIELDS' in line:
                fields = line.split()[1:]
            elif 'DATA' in line:
                data_type = line.split()[1]
                break

        # Read each point
        points = []
        while True:
            point_data = {}
            for i, field in enumerate(fields):
                # Choose byte‐length & unpack code
                if "xt32" in filename:
                    num_bytes = hesai_unpack_numbytes[i]
                    code      = hesai_fieldtypecodes[i]
                else:
                    num_bytes = 4
                    code      = 'f'
                buf = file.read(num_bytes)
                if not buf:
                    return fields, points
                point_data[field] = struct.unpack(code, buf)[0]
            points.append([point_data[f] for f in fields])

def load_pcd(filename):
    """
    Returns an (N×3) float32 array of [x,y,z] by leveraging read_pcd_file.
    """
    fields, pts_list = read_pcd_file(filename)
    pts = np.array(pts_list, dtype=np.float32)
    ix, iy, iz = fields.index('x'), fields.index('y'), fields.index('z')
    return pts[:, [ix, iy, iz]]

def pos_interpolate(gt_t, gt_p, query_t):
    out = np.zeros((len(query_t), 3))
    for i in range(3):
        out[:, i] = np.interp(query_t, gt_t, gt_p[:, i])
    return out

def rot_slerp(gt_t, gt_q, query_t):
    slerp = Slerp(gt_t, R.from_quat(gt_q))
    return slerp(query_t).as_quat()

def interpolation(gt_t, gt_p, gt_q, query_t):
    """Return list of 4×4 LiDAR→world poses at each time."""
    positions = pos_interpolate(gt_t, gt_p, query_t)
    quats     = rot_slerp(gt_t, gt_q, query_t)
    poses = []
    for pos, quat in zip(positions, quats):
        T = np.eye(4)
        T[:3, :3] = R.from_quat(quat).as_matrix()
        T[:3,  3] = pos
        poses.append(T)
    return poses

def extract_ts(fname):
    return float(os.path.splitext(fname)[0])




## Visualize Point Clouds

Change `start_ts` and `end_ts` to specify time period to visalize.
Change `base_dir` to specify dataset path.

In [ ]:
base_dir         = "/datasets/snail-radar/bc/20230920_1"
lidar_dir        = f"{base_dir}/xt32"
radar_dir        = f"{base_dir}/eagleg7/enhanced"
pose_file        = f"{base_dir}/utm50r_T_xt32.txt"
lidar_calib_path = f"{base_dir}/body_T_xt32.txt"
radar_calib_path = f"{base_dir}/body_T_oculii.txt"

start_ts, end_ts = 1695213782.0, 1695213800.0

# -- 1. Load ground-truth LiDAR poses ---------------------------------------
gt_t, gt_p, gt_q = [], [], []
with open(pose_file) as f:
    for line in f:
        parts = line.split()
        if len(parts) < 8 or parts[0].startswith('#'):
            continue
        gt_t.append(float(parts[0]))
        gt_p.append(list(map(float, parts[1:4])))
        gt_q.append(list(map(float, parts[4:8])))
gt_t = np.array(gt_t); gt_p = np.array(gt_p); gt_q = np.array(gt_q)

# -- 2. Compute LiDAR→Radar extrinsic ----------------------------------------
Body_T_L = np.loadtxt(lidar_calib_path).reshape(4,4)
Body_T_R = np.loadtxt(radar_calib_path).reshape(4,4)
L_T_R    = np.linalg.inv(Body_T_L) @ Body_T_R

# -- 3. Prepare sorted LiDAR & Radar lists ----------------------------------
lidar_files = sorted(f for f in os.listdir(lidar_dir) if f.endswith('.pcd'))
lidar_times = np.array([extract_ts(f) for f in lidar_files])
lidar_mask  = (lidar_times >= start_ts) & (lidar_times <= end_ts)
sel_lidar   = [lidar_files[i] for i in np.where(lidar_mask)[0]]
sel_l_ts    = lidar_times[lidar_mask]
lidar_poses = interpolation(gt_t, gt_p, gt_q, sel_l_ts)

radar_files = sorted(f for f in os.listdir(radar_dir) if f.endswith('.pcd'))
radar_times = np.array([extract_ts(f) for f in radar_files])
radar_mask  = (radar_times >= start_ts) & (radar_times <= end_ts)
sel_radar   = [radar_files[i] for i in np.where(radar_mask)[0]]
sel_r_ts    = radar_times[radar_mask]
radar_poses_lidar = interpolation(gt_t, gt_p, gt_q, sel_r_ts)

# convert LiDAR→world poses to Radar→world
radar_poses = [U_T_L @ L_T_R for U_T_L in radar_poses_lidar]

# -- 4. Visualization: match each radar frame to closest LiDAR frame -------
rr.init("viz_frame_sync", spawn=True)
rr.notebook_show(width=1200, height=800)

for step, (rfile, U_T_R, r_ts) in enumerate(zip(sel_radar, radar_poses, sel_r_ts)):
    # find nearest LiDAR index
    idx = np.argmin(np.abs(sel_l_ts - r_ts))
    lidar_file = sel_lidar[idx]
    U_T_L      = lidar_poses[idx]
    # load and transform that LiDAR scan
    pts_l = load_pcd(os.path.join(lidar_dir, lidar_file))
    pts_l_h = np.hstack([pts_l, np.ones((len(pts_l),1))])
    world_l = (U_T_L @ pts_l_h.T).T[:, :3]

    # load and transform radar scan
    pts_r = load_pcd(os.path.join(radar_dir, rfile))
    pts_r_h = np.hstack([pts_r, np.ones((len(pts_r),1))])
    world_r = (U_T_R @ pts_r_h.T).T[:, :3]

    # log per-frame
    rr.set_time_sequence("step", step)
    rr.log("lidar", rr.Points3D(world_l, colors=[0.0,0.0,1.0], radii=0.1))
    rr.log("radar", rr.Points3D(world_r, colors=[1.0,0.0,0.0], radii=0.1))